In [72]:
from pymongo import MongoClient
import operator

client = MongoClient('127.0.0.1', 27017)
db = client.influencers_db

categories = ['Technology', 'Food', 'Gaming', 'Beauty', 'Entertainment', 'Fashion', 'Home', 'Fitness', 'Travel']

kl_scores = []
rf_scores = []

# fb_followers_wt = 7
# ig_followers_wt = 7
# tw_followers_wt = 9
# yt_followers_wt = 12
# ig_popularity_wt = 6.25
# tw_popularity_wt = 8.75
# yt_popularity_wt = 10
# fb_interaction_wt = 11
# ig_interaction_wt = 13
# yt_interaction_wt = 16

fb_followers_wt = 5
ig_followers_wt = 22.5
tw_followers_wt = 27.5
yt_followers_wt = 15
ig_popularity_wt = 5
tw_popularity_wt = 6
yt_popularity_wt = 4
fb_interaction_wt = 4
ig_interaction_wt = 8
yt_interaction_wt = 4

for category in categories:
    fb_followers_sum = 0
    ig_followers_sum = 0
    tw_followers_sum = 0
    yt_followers_sum = 0
    ig_popularity_sum = 0
    tw_popularity_sum = 0
    yt_popularity_sum = 0
    fb_interaction_sum = 0
    ig_interaction_sum = 0
    yt_interaction_sum = 0

    all_inf_data = db.influencers_list_collection.find({'category': category})
    for inf in all_inf_data:
        tw_handle = inf['tw_handle']

        details = db.combined_collection.find_one({'tw_handle': tw_handle})
        screen_name = details['tw_screen_name']
        
        # Popularity
        try:
            ig_avg_likes = db.instagram_collection.aggregate([{'$match': {'twitter_handle': {'$eq': tw_handle}}}, {
                                                             '$group': {'_id': '$twitter_handle', 'average': {'$avg': '$likes'}}}])
            ig_popularity = list(ig_avg_likes)[0]['average']
            ig_avg_comments = db.instagram_collection.aggregate([{'$match': {'twitter_handle': {'$eq': tw_handle}}}, {
															'$group': {'_id': '$twitter_handle', 'average': {'$avg': '$comments'}}}])
            ig_interaction = list(ig_avg_comments)[0]['average']
            ig_followers = db.instagram_followers_collection.find_one({'twitter': tw_handle})['follower']
        except:
            ig_popularity = 0
            ig_interaction = 0
            ig_followers = 0

        try:
            tw_avg_likes = db.twitter_collection.aggregate([{'$match': {'screen_name': {'$eq': screen_name}}}, {
                                                       '$group': {'_id': '$screen_name', 'average': {'$avg': '$favorites'}}}])
            tw_popularity = list(tw_avg_likes)[0]['average']
            tw_followers = details['tw_followers_count']

        except:
            tw_popularity = 0
        
        try:
            yt_avg_likes = db.youtube_collection.aggregate([{'$match': {'twitter_handle': {'$eq': tw_handle}}}, {
                                                           '$group': {'_id': '$twitter_handle', 'average': {'$avg': '$likes'}}}])
            yt_popularity = list(yt_avg_likes)[0]['average']
#             print(tw_handle, yt_popularity)
            yt_avg_comments = db.youtube_collection.aggregate([{'$match': {'twitter_handle': {'$eq': tw_handle}}}, {
														  '$group': {'_id': '$twitter_handle', 'average': {'$avg': '$comments'}}}])
            yt_interaction = list(yt_avg_comments)[0]['average']
            yt_followers = details['yt_subscriber_count']
        except:
            yt_popularity = 0
            yt_interaction = 0
            yt_followers = 0
        
        try:
            fb_avg_comments = db.facebook_new_collection.aggregate([{'$match': {'twitter_handle': {'$eq': tw_handle}}}, {
                                                                   '$group': {'_id': '$twitter_handle', 'average': {'$avg': '$fb_no_of_comments'}}}])
            fb_interaction = list(fb_avg_comments)[0]['average']
            fb_followers = details['fb_page_likes']
        except:
            fb_interaction = 0
            fb_followers = 0
        
        fb_followers_sum += fb_followers
        ig_followers_sum += ig_followers
        tw_followers_sum += tw_followers
        yt_followers_sum += yt_followers
        ig_popularity_sum += ig_popularity
        tw_popularity_sum += tw_popularity
        yt_popularity_sum += yt_popularity
        fb_interaction_sum += fb_interaction
        ig_interaction_sum += ig_interaction
        yt_interaction_sum += yt_interaction

#     print(fb_followers_sum, ig_followers_sum, tw_followers_sum, yt_followers_sum)
#     print(ig_popularity_sum, tw_popularity_sum, yt_popularity_sum)
#     print(fb_interaction_sum, ig_interaction_sum, yt_interaction_sum)

    rf_score_dict = dict()

    all_inf_data = db.influencers_list_collection.find({'category': category})
    for inf in all_inf_data:
        tw_handle = inf['tw_handle']

        details = db.combined_collection.find_one({'tw_handle': tw_handle})
        screen_name = details['tw_screen_name']
        
        # Popularity
        try:
            ig_avg_likes = db.instagram_collection.aggregate([{'$match': {'twitter_handle': {'$eq': tw_handle}}}, {
                                                             '$group': {'_id': '$twitter_handle', 'average': {'$avg': '$likes'}}}])
            ig_popularity = list(ig_avg_likes)[0]['average']
            ig_avg_comments = db.instagram_collection.aggregate([{'$match': {'twitter_handle': {'$eq': tw_handle}}}, {
															'$group': {'_id': '$twitter_handle', 'average': {'$avg': '$comments'}}}])
            ig_interaction = list(ig_avg_comments)[0]['average']
            ig_followers = db.instagram_followers_collection.find_one({'twitter': tw_handle})['follower']
        except:
            ig_popularity = 0
            ig_interaction = 0
            ig_followers = 0

        try:
            tw_avg_likes = db.twitter_collection.aggregate([{'$match': {'screen_name': {'$eq': screen_name}}}, {
                                                       '$group': {'_id': '$screen_name', 'average': {'$avg': '$favorites'}}}])
            tw_popularity = list(tw_avg_likes)[0]['average']
            tw_followers = details['tw_followers_count']

        except:
            tw_popularity = 0
        
        try:
            yt_avg_likes = db.youtube_collection.aggregate([{'$match': {'twitter_handle': {'$eq': tw_handle}}}, {
                                                           '$group': {'_id': '$twitter_handle', 'average': {'$avg': '$likes'}}}])
            yt_popularity = list(yt_avg_likes)[0]['average']
            yt_avg_comments = db.youtube_collection.aggregate([{'$match': {'twitter_handle': {'$eq': tw_handle}}}, {
														  '$group': {'_id': '$twitter_handle', 'average': {'$avg': '$comments'}}}])
            yt_interaction = list(yt_avg_comments)[0]['average']
            yt_followers = details['yt_subscriber_count']
        except:
            yt_popularity = 0
            yt_interaction = 0
            yt_followers = 0
        
        try:
            fb_avg_comments = db.facebook_new_collection.aggregate([{'$match': {'twitter_handle': {'$eq': tw_handle}}}, {
                                                                   '$group': {'_id': '$twitter_handle', 'average': {'$avg': '$fb_no_of_comments'}}}])
            fb_interaction = list(fb_avg_comments)[0]['average']
            fb_followers = details['fb_page_likes']
        except:
            fb_interaction = 0
            fb_followers = 0
        
        fb_followers_prop_val = (fb_followers/fb_followers_sum)*fb_followers_wt
        ig_followers_prop_val = (ig_followers/ig_followers_sum)*ig_followers_wt
        tw_followers_prop_val = (tw_followers/tw_followers_sum)*tw_followers_wt
        yt_followers_prop_val = (yt_followers/yt_followers_sum)*yt_followers_wt
        ig_popularity_prop_val = (ig_popularity/ig_popularity_sum)*ig_popularity_wt
        tw_popularity_prop_val = (tw_popularity/tw_popularity_sum)*tw_popularity_wt
        yt_popularity_prop_val = (yt_popularity/yt_popularity_sum)*yt_popularity_wt
        fb_interaction_prop_val = (fb_interaction/fb_interaction_sum)*fb_interaction_wt
        ig_interaction_prop_val = (ig_interaction/ig_interaction_sum)*ig_interaction_wt
        yt_interaction_prop_val = (yt_interaction/yt_interaction_sum)*yt_interaction_wt
        
#         print(fb_followers_prop_val , ig_followers_prop_val , tw_followers_prop_val , \
#                     yt_followers_prop_val , ig_popularity_prop_val , tw_popularity_prop_val , \
#                     yt_popularity_prop_val , fb_interaction_prop_val , ig_interaction_prop_val , \
#                     yt_interaction_prop_val)
#         print(yt_popularity_prop_val, yt_popularity, yt_popularity_sum)
        
        rf_score = fb_followers_prop_val + ig_followers_prop_val + tw_followers_prop_val + \
                    yt_followers_prop_val + ig_popularity_prop_val + tw_popularity_prop_val + \
                    yt_popularity_prop_val + fb_interaction_prop_val + ig_interaction_prop_val + \
                    yt_interaction_prop_val
#         import math
#         if not math.isnan(rf_score):
#             kl_scores.append(float(details['kl_score']))
#             rf_scores.append(float(rf_score))
        rf_score_dict[tw_handle] = rf_score
    sorted_rf_score_dict = dict(sorted(rf_score_dict.items(), key=operator.itemgetter(1), reverse=True))
    rank_dict = dict()
    rank = 1
    for key, value in sorted_rf_score_dict.items():
        rank_dict[key] = rank
        rank += 1
    print(sorted_rf_score_dict)
    print(rank_dict)
    
    for key, value in sorted_rf_score_dict.items():
        db.rightfluencer_collection.insert_one({'tw_handle': key, 'category': category,
                                                'rf_score': sorted_rf_score_dict[key], 'rf_rank': rank_dict[key]})

    
    
# from scipy.stats.stats import pearsonr, spearmanr
# print(spearmanr(kl_scores, rf_scores), pearsonr(kl_scores, rf_scores))

{'MKBHD': 25.52634023479844, 'UnboxTherapy': 20.527791828231653, 'ijustine': 16.78592872398491, 'AndroidAuth': 8.584527927391903, 'LinusTech': 7.865647092531321, 'austinnotduncan': 5.878885472230412, 'tldtoday': 5.620416987756167, 'GuyKawasaki': 4.818316381325086, 'TechnoBuffalo': 3.549746727147683, 'Dave2Dtv': 1.84239862460242}
{'MKBHD': 1, 'UnboxTherapy': 2, 'ijustine': 3, 'AndroidAuth': 4, 'LinusTech': 5, 'austinnotduncan': 6, 'tldtoday': 7, 'GuyKawasaki': 8, 'TechnoBuffalo': 9, 'Dave2Dtv': 10}
{'rosannapansino': 50.71096879203326, 'yolanda_gampp': 13.86967448609904, 'CupcakeAddictAU': 10.256833241272803, 'fitmencook': 6.239298218911132, 'sweetambs': 6.155448937347156, 'smittenkitchen': 4.757701888624631, 'NatashasKitchen': 3.958701997374101, 'joythebaker': 2.824597476293415, 'inspiralized': 1.2255120592416215, 'izyhossack': 1.0012629028028404}
{'rosannapansino': 1, 'yolanda_gampp': 2, 'CupcakeAddictAU': 3, 'fitmencook': 4, 'sweetambs': 5, 'smittenkitchen': 6, 'NatashasKitchen': 7, 